# Recommendation using Apriori Algorithm

Applying Apriori algorithm to recommend something

Dataset used from [Kaggle](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset)

In [2]:
# importing library
import pandas as pd

# Open the dataset
# -----------------------
#Table 1
books = pd.read_csv('/Users/patriciasilva/Downloads/Preditiva/archive/Books.csv')
# books

# -----------------------
#Table 2
ratings = pd.read_csv('/Users/patriciasilva/Downloads/Preditiva/archive/Ratings.csv')
# ratings

# -----------------------
#Table 3
users = pd.read_csv('/Users/patriciasilva/Downloads/Preditiva/archive/Users.csv')
# users

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/1853337994.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/Users/patriciasilva/Downloads/Preditiva/archive/Books.csv')


### Cleaning the dataset

In [14]:
import re

# verifying the dataset
df = input('Which dataset do you want to verify (books, ratings or users)? ')

# Check the user's choice and perform actions accordingly
if df == 'books':
    # Code for dataset 1
    df = books
    
    # Function to clean unwanted characters from string columns
    def clean_string_column(column):
        cleaned_column = column.apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
        cleaned_column = cleaned_column.apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
        return cleaned_column
    
    # Clean the dataset
    cleaned_dataset = df.copy()
    string_columns = ['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']
    for column in string_columns:
        cleaned_dataset[column] = clean_string_column(df[column])
    
    cleaned_dataset = cleaned_dataset[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
    cleaned_dataset.rename(columns = {'Book-Title':'Book_Title', 'Book-Author':'Book_Author', 'Year-Of-Publication':'Year_Of_Publication'}, inplace = True)
    #cleaned_dataset.info()
    
    # Saving books dataset
    cleaned_dataset.to_csv('/Users/patriciasilva/Downloads/Preditiva/archive/Books_cleaned.csv', index=False)

elif df == 'ratings':
    # Code for dataset 2
    df = ratings
    df.rename(columns = {'User-ID':'UserID', 'Book-Rating':'Book_Rating'}, inplace = True)
    
    # Saving ratings dataset
    cleaned_dataset.to_csv('/Users/patriciasilva/Downloads/Preditiva/archive/Ratings_cleaned2.csv', index=False)

elif df == 'users':
    # Code for dataset 3
    df = users
    
    # Drop null values on Table 3
    df = df.dropna()
    
    # Function to clean unwanted characters from string columns
    def clean_string_column(column):
        cleaned_column = column.apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
        cleaned_column = cleaned_column.apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
        return cleaned_column
    
    # Clean the dataset
    cleaned_dataset = df.copy()

    # Separate location information into city, state, and country columns
    location_column = df['Location']
    location_split = location_column.str.split(',', expand=True).rename(columns={0: 'City', 1: 'State', 2: 'Country'})
    cleaned_dataset = pd.concat([cleaned_dataset, location_split], axis=1)

    
    # Function to clean unwanted characters from string columns
    def clean_string_column(column):
        cleaned_column = column.apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
        cleaned_column = cleaned_column.apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
        return cleaned_column

    # Clean book title, book author, and publisher columns
    string_columns = ['City', 'State', 'Country']
    for column in string_columns:
        cleaned_dataset[column] = clean_string_column(cleaned_dataset[column])
    
    cleaned_dataset = cleaned_dataset[['User-ID','Age', 'City', 'State', 'Country']]
    cleaned_dataset.rename(columns = {'User-ID':'UserID'}, inplace = True)
    #cleaned_dataset.info()
    
    # Saving users dataset
    cleaned_dataset.to_csv('/Users/patriciasilva/Downloads/Preditiva/archive/Users_cleaned2.csv', index=False)
    

else:
    # Invalid choice
    print("Invalid dataset choice. Please choose between books, ratings or users.")

# df.info(verbose=True, show_counts=True)


# import seaborn as sns

# #Veriffying NaN values
# sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Which dataset do you want to verify (books, ratings or users)? users


### Applying Apriori

First the cleaned datasets from books, ratings and users were joined on MySQL and saved to be user below

In [15]:
# Open table joined in MySQL
import pandas as pd

books_df = pd.read_csv('/Users/patriciasilva/Downloads/Preditiva/archive/all_table_joined.csv')
#books_df

# Fill NaN to zero
books_df = books_df.fillna(0)

#books_df.info()

In [ ]:
# Installing efficient apriori
#!pip install efficient-apriori

### Two ways to used:

1) Analyse Support and Confidence data on Python (first cell below)
2) Import to an excel spreadsheet the following values: Antecedent, Consequent, Support and Confidence (second cell below)

In [16]:
# ============= 1) Analyse Support and Confidence data on Python =================
# Importing libraries
from efficient_apriori import apriori
import pandas as pd

# # ----------- All data --------------
# Convert numeric columns to string
numeric_cols = ['UserID', 'Age', 'Book_Rating', 'Year_Of_Publication']
books_df[numeric_cols] = books_df[numeric_cols].astype(str)

# Convert dataset to a list of lists
transactions = books_df.values.tolist()

# Apply the Apriori algorithm
itemsets, rules = apriori(transactions, min_support=0.01, min_confidence=0.6)

# Print the association rules
for rule in rules:
    print(rule)

    
# # ----------- Two variables (Age and Book Title) --------------
# # Convert numeric columns to string
# numeric_cols = ['Age']
# books_df[numeric_cols] = books_df[numeric_cols].astype(str)

# books_age = books_df[['Age', 'Book_Title']]

# # Convert dataset to a list of lists
# transactions = books_age.values.tolist()

# # Apply the Apriori algorithm
# itemsets, rules = apriori(transactions, min_support=0.001, min_confidence=0.5)

# # Print the association rules
# for rule in rules:
#     print(rule)
    

# # ----------- Two variables (Book Rating and Book Author) --------------
# # Convert numeric columns to string
# numeric_cols = ['Book_Rating']
# books_df[numeric_cols] = books_df[numeric_cols].astype(str)

# books_rat_aut = books_df[['Book_Rating', 'Book_Author']]

# # Convert dataset to a list of lists
# transactions = books_rat_aut.values.tolist()

# # Apply the Apriori algorithm
# itemsets, rules = apriori(transactions, min_support=0.01, min_confidence=0.5)

# # Print the association rules
# for rule in rules:
#     print(rule)
    

# ----------- Two variables (User ID and Age) --------------
# # Convert numeric columns to string
# numeric_cols = ['UserID', 'Age']
# books_df[numeric_cols] = books_df[numeric_cols].astype(str)

# books_user_age = books_df[['Book_Author','UserID', 'Age']]

# # Convert dataset to a list of lists
# transactions = books_user_age.values.tolist()

# # Apply the Apriori algorithm
# itemsets, rules = apriori(transactions, min_support=0.001, min_confidence=0.5)

# # Print the association rules
# for rule in rules:
#     print(rule)

{0971880107} -> {0} (conf: 1.000, supp: 0.010, lift: 1.776, conv: 437000000.000)
{1983} -> {0} (conf: 0.909, supp: 0.010, lift: 1.615, conv: 4.807)
{1989} -> {0} (conf: 0.810, supp: 0.017, lift: 1.438, conv: 2.294)
{1990} -> {0} (conf: 0.750, supp: 0.018, lift: 1.332, conv: 1.748)
{1991} -> {0} (conf: 0.650, supp: 0.013, lift: 1.155, conv: 1.249)
{1992} -> {0} (conf: 0.621, supp: 0.018, lift: 1.102, conv: 1.152)
{2000} -> {0} (conf: 0.602, supp: 0.050, lift: 1.070, conv: 1.099)
{22} -> {0} (conf: 0.879, supp: 0.029, lift: 1.561, conv: 3.605)
{276925} -> {0} (conf: 0.867, supp: 0.026, lift: 1.539, conv: 3.277)
{277195} -> {0} (conf: 0.884, supp: 0.038, lift: 1.570, conv: 3.758)
{277378} -> {0} (conf: 0.792, supp: 0.019, lift: 1.406, conv: 2.098)
{277427} -> {0} (conf: 0.744, supp: 0.145, lift: 1.321, conv: 1.704)
{277466} -> {0} (conf: 0.714, supp: 0.010, lift: 1.269, conv: 1.529)
{277478} -> {0} (conf: 0.878, supp: 0.036, lift: 1.560, conv: 3.583)
{277639} -> {0} (conf: 0.700, supp: 0.

In [18]:
# ============= 2) Import to an excel spreadsheet =================
from efficient_apriori import apriori
import pandas as pd


# Convert numeric columns to string
numeric_cols = ['UserID', 'Age', 'Book_Rating', 'Year_Of_Publication']
books_df[numeric_cols] = books_df[numeric_cols].astype(str)

# Convert dataset to a list of lists
transactions = books_df.values.tolist()

# Apply the Apriori algorithm
itemsets, rules = apriori(transactions, min_support=0.01, min_confidence=0.5)

# Create a DataFrame to store the rules
rules_df = pd.DataFrame(columns=['Antecedent', 'Consequent', 'Support', 'Confidence'])

# Store the rules in the DataFrame
for rule in rules:
    antecedent = ', '.join(rule.lhs)
    consequent = ', '.join(rule.rhs)
    support = rule.support
    confidence = rule.confidence
    
    rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
                                'Support': support, 'Confidence': confidence}, ignore_index=True)

# Save the rules DataFrame to a spreadsheet
rules_df.to_excel('/Users/patriciasilva/Downloads/Preditiva/archive/association_rules2.xlsx', index=False)

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append

/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rules_df = rules_df.append({'Antecedent': antecedent, 'Consequent': consequent,
/var/folders/nm/l3pjj45d1759l0k7vj10p7gw0000gn/T/ipykernel_2921/372584880.py:26: FutureWarning: The frame.append